## Difference-in-differences (DID) in Economics

DiD is one of the quasi-experimental statistical techniques methods widely used in econometrics and quantitative research. In this method, observational data is used. Quasi experimental methods are crucial in estimating causal effects considering a set of assumptions, especially when randomization is not possible.

In this notebook, I try to perform DiD analysis using a fictional dataset provided during the lecture. This notebook is based on the lectures on "Quasi-Experiments in Development Economics" by Prof. Dr. Sebastian Vollmer at the University of Göttingen.

https://flexnow2.uni-goettingen.de/modulbeschreibungen/66723.pdf

DiD is only implemented in panel data, with observation from multiple time periods and multiple units. In this notebook, I implement DiD for multiple datasets.

### 1. First DiD Analysis

Based on a dataset on 7 countries

In [1]:
# library
library(foreign)
library(tidyverse)
data1 <- read.dta("http://dss.princeton.edu/training/Panel101.dta")

# Check out the data
dim(data1)

# check out in a bit detail
glimpse(data1)


Warning message:
"Paket 'tidyverse' wurde unter R Version 4.1.3 erstellt"
-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.2 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.6     v dplyr   1.0.7
v tidyr   1.1.4     v stringr 1.4.0
v readr   2.1.3     v forcats 0.5.1

Warning message:
"Paket 'readr' wurde unter R Version 4.1.3 erstellt"
-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



[1] 70  9

Rows: 70
Columns: 9
$ country <fct> A, A, A, A, A, A, A, A, A, A, B, B, B, B, B, B, B, B, B, B, C,~
$ year    <int> 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 19~
$ y       <dbl> 1342787840, -1899660544, -11234363, 2645775360, 3008334848, 32~
$ y_bin   <dbl> 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,~
$ x1      <dbl> 0.27790365, 0.32068470, 0.36346573, 0.24614404, 0.42462304, 0.~
$ x2      <dbl> -1.1079559, -0.9487200, -0.7894840, -0.8855330, -0.7297683, -0~
$ x3      <dbl> 0.28255358, 0.49253848, 0.70252335, -0.09439092, 0.94613063, 1~
$ opinion <fct> Str agree, Disag, Disag, Disag, Disag, Str agree, Disag, Str a~
$ op      <dbl> 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1,~


In [2]:
head(data1)

,country,year,y,y_bin,x1,x2,x3,opinion,op
,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>
1,A,1990,1342787840,1,0.2779036,-1.1079559,0.28255358,Str agree,1
2,A,1991,-1899660544,0,0.3206847,-0.9487200,0.49253848,Disag,0
3,A,1992,-11234363,0,0.3634657,-0.7894840,0.70252335,Disag,0
4,A,1993,2645775360,1,0.2461440,-0.8855330,-0.09439092,Disag,0
5,A,1994,3008334848,1,0.4246230,-0.7297683,0.94613063,Disag,0
6,A,1995,3229574144,1,0.4772141,-0.7232460,1.02968037,Str agree,1


The data above consists of 7 countries named from A to G with 10 years of observation fom the year 1990 to 1999. Thus, the dimension of the dataset is *70X9*.

*Suppose some kind of intervention began in the year 1994, for countries E, F, G. However, the remaining countries A, B, C, and D didn't receive any kind of intervention.*

In [3]:
attach(data1)
D_i <- ifelse(data1$year >= 1994, 1, 0) # timing dummy

Post_t <- ifelse(data1$country == "E" | data1$country == "F" |
                       data1$country == "G", 1, 0) # treatment dummy



#### Relevant DiD Regression Equation

$Y_{it} = \beta_0 + \beta_1D_i + \beta_2Post_t + \beta_3D_i*Post_t + \epsilon_{it}$

For untreated before 1994: $Y_{it} = \beta_0 + \epsilon_{it}$

For untreated after 1994: $Y_{it} = \beta_0 + \beta_1 + \epsilon_{it}$

For treated before 1994: $ Y_{it} = \beta_0 + \beta_2 + \epsilon_{it}$

For treated after 1994: $Y_{it} = \beta_0 + \beta_1 + \beta_2 + \beta_3 + \epsilon_{it}$
